## Explore and label data

This notebook uses the ```napari-easy-augment-batch-dl``` widget to explore and label the data.  If we have a model trained we can also predict using that model. 

In [ ]:
import napari
viewer = napari.Viewer()

import os
import numpy as np
import napari
from napari_easy_augment_batch_dl import easy_augment_batch_dl

# import the frameworks you want to use.  As part of the import the framework wil be registered
# if it calls the BaseFramework.register_framework() method
try:
    from napari_easy_augment_batch_dl.frameworks.pytorch_semantic_framework import PytorchSemanticFramework
except:
    print('PytorchSemanticFramework not loaded')
try:
    from napari_easy_augment_batch_dl.frameworks.cellpose_instance_framework import CellPoseInstanceFramework
except:
    print('CellPoseInstanceFramework not loaded')
try:
    from napari_easy_augment_batch_dl.frameworks.stardist_instance_framework import StardistInstanceFramework
except:
    print('StardistInstanceFramework not loaded')

## Label the Image with a Napari widget

Here we call the ```napari-easy-augment-batch-dl``` widget in ```label_only``` mode so we can label the images.   Optionally if a model has already been trained we can pass it to the Widget.  Then we can make predictions and correct them. 

In [ ]:


batch_dl = easy_augment_batch_dl.NapariEasyAugmentBatchDL(viewer, label_only = False)

viewer.window.add_dock_widget(
    batch_dl
)

data_path = r'../../data'
parent_path = os.path.join(data_path, 'ladybugs1')
model_path = os.path.join(parent_path, 'models')

batch_dl.load_image_directory(parent_path)

model_name = None
model_name = 'stardist_for_ladybugs'
#model_name = 'cellpose_for_ladybugs'


if model_name == 'stardist_for_ladybugs':
    # set the drop down to the StardistInstanceFramework
    batch_dl.network_architecture_drop_down.setCurrentText(StardistInstanceFramework.descriptor)

    # get the stardist widget
    widget = batch_dl.deep_learning_widgets[StardistInstanceFramework.descriptor]

    # load model
    widget.load_model_from_path(os.path.join(model_path, model_name))

elif model_name == 'cellpose_for_ladybugs':
    # set the drop down to the CellPoseInstanceFramework
    batch_dl.network_architecture_drop_down.setCurrentText(CellPoseInstanceFramework.descriptor)
    
    # get the cellpose widget
    widget = batch_dl.deep_learning_widgets[CellPoseInstanceFramework.descriptor]
    
    # load model
    widget.load_model_from_path(os.path.join(model_path, model_name))

    # get the framework object
    framework = batch_dl.deep_learning_project.frameworks[CellPoseInstanceFramework.descriptor]

    # set the parameters for the cellpose framework
    framework.prob_thresh = -1
    framework.flow_thresh = 0.4
    framework.chan_segment = 0 
    framework.chan2 = 1

    # sync parameters on the widget after this we should see the widget in Napari synced with the parameters we set on the framework
    widget.sync_with_framework()
   